In [ ]:
!pip install pyspark
!apt-get install openjdk-8-jdk-headless -qq

##Drive on

In [ ]:
import pyspark
import numpy as np
import pandas as pd
import os

from pyspark.context import SparkContext
from pyspark.ml import Pipeline
from pyspark.sql import SparkSession
from pyspark.sql.functions import lit, col, broadcast
from pyspark.sql import functions as F
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler
from pyspark.sql.functions import udf
from pyspark.sql.types import FloatType
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

from tqdm.notebook import tqdm

In [ ]:
import os
from google.colab import drive
drive.mount('/content/gdrive')

path = "gdrive/MyDrive/Final Data/"
PATH = "gdrive/MyDrive/추천시스템/정리/data/"
os.listdir(path)

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


['1210 all_pp.csv',
 '1210 CosSim_All.csv',
 '1210 CosSim_Title.csv',
 '1210 CosSim_Genres.csv',
 '1210 CosSim_Tag.csv',
 '1210 CosSim_Story.csv',
 '1210 EucSim_Story.csv',
 '1210 EucSim_Tag.csv',
 '1210 EucSim_Genres.csv',
 '1210 EucSim_Title.csv',
 '1210 EucSim_All.csv',
 'df_pred_euc_tag_sp.csv',
 'df_pred_euc_title_sp.csv',
 'df_pred_euc_all_sp.csv',
 'df_pred_euc_story_sp.csv',
 'df_pred_cos_all_sp.csv']

In [ ]:
cosSim_All    = pd.read_csv(path + '1210 CosSim_All.csv')
cosSim_Genres = pd.read_csv(path + '1210 CosSim_Genres.csv')
cosSim_Title  = pd.read_csv(path + '1210 CosSim_Title.csv')
cosSim_Tag    = pd.read_csv(path + '1210 CosSim_Tag.csv')
cosSim_Story  = pd.read_csv(path + '1210 CosSim_Story.csv')

In [ ]:
EucSim_All    =  pd.read_csv(path + '1210 EucSim_All.csv')
EucSim_Genres = pd.read_csv(path + '1210 EucSim_Genres.csv')
EucSim_Title  = pd.read_csv(path + '1210 EucSim_Title.csv')
EucSim_Tag    = pd.read_csv(path + '1210 EucSim_Tag.csv')
EucSim_Story  = pd.read_csv(path + '1210 EucSim_Story.csv')

In [ ]:
k = pd.read_csv(path + "1210 all_pp.csv")
movie = pd.read_csv(PATH + "movies_refined.csv")
rating = pd.read_csv(PATH + 'ratings_refined.csv')

##Merging and preprocessing files

In [ ]:
new_data = pd.merge(movie, k, on='movieId', how='right')

In [ ]:
new_df = pd.merge(movie, k, on='movieId', how='left')

In [ ]:
new_ratings = pd.merge(rating, k, on='movieId', how='left')

In [ ]:
title_list = new_data['title_x'].tolist()

In [ ]:
movie_id_list = new_data['movieId'].tolist()

###Set Euclidean matrix index

In [ ]:
EucSim_All_id    = EucSim_All.set_index(pd.Index(movie_id_list)).rename(columns=dict(zip(EucSim_All.columns, movie_id_list)))
EucSim_Genres_id = EucSim_Genres.set_index(pd.Index(movie_id_list)).rename(columns=dict(zip(EucSim_Genres.columns, movie_id_list)))
EucSim_Title_id  = EucSim_Title.set_index(pd.Index(movie_id_list)).rename(columns=dict(zip(EucSim_Title.columns, movie_id_list)))
EucSim_Tag_id    = EucSim_Tag.set_index(pd.Index(movie_id_list)).rename(columns=dict(zip(EucSim_Tag.columns, movie_id_list)))
EucSim_Story_id  = EucSim_Story.set_index(pd.Index(movie_id_list)).rename(columns=dict(zip(EucSim_Story.columns, movie_id_list)))

In [ ]:
EucSim_All    = EucSim_All.set_index(pd.Index(title_list)).rename(columns=dict(zip(EucSim_All.columns, title_list)))
EucSim_Genres = EucSim_Genres.set_index(pd.Index(title_list)).rename(columns=dict(zip(EucSim_Genres.columns, title_list)))
EucSim_Title  = EucSim_Title.set_index(pd.Index(title_list)).rename(columns=dict(zip(EucSim_Title.columns, title_list)))
EucSim_Tag    = EucSim_Tag.set_index(pd.Index(title_list)).rename(columns=dict(zip(EucSim_Tag.columns, title_list)))
EucSim_Story  = EucSim_Story.set_index(pd.Index(title_list)).rename(columns=dict(zip(EucSim_Story.columns, title_list)))

###Set cosine matrix index

In [ ]:
cosSim_All_id    = cosSim_All.set_index(pd.Index(movie_id_list)).rename(columns=dict(zip(cosSim_All.columns, movie_id_list)))
cosSim_Genres_id = cosSim_Genres.set_index(pd.Index(movie_id_list)).rename(columns=dict(zip(cosSim_Genres.columns, movie_id_list)))
cosSim_Title_id  = cosSim_Title.set_index(pd.Index(movie_id_list)).rename(columns=dict(zip(cosSim_Title.columns, movie_id_list)))
cosSim_Tag_id    = cosSim_Tag.set_index(pd.Index(movie_id_list)).rename(columns=dict(zip(cosSim_Tag.columns, movie_id_list)))
cosSim_Story_id  = cosSim_Story.set_index(pd.Index(movie_id_list)).rename(columns=dict(zip(cosSim_Story.columns, movie_id_list)))

In [ ]:
cosSim_All    = cosSim_All.set_index(pd.Index(title_list)).rename(columns=dict(zip(cosSim_All.columns, title_list)))
cosSim_Genres = cosSim_Genres.set_index(pd.Index(title_list)).rename(columns=dict(zip(cosSim_Genres.columns, title_list)))
cosSim_Title  = cosSim_Title.set_index(pd.Index(title_list)).rename(columns=dict(zip(cosSim_Title.columns, title_list)))
cosSim_Tag    = cosSim_Tag.set_index(pd.Index(title_list)).rename(columns=dict(zip(cosSim_Tag.columns, title_list)))
cosSim_Story  = cosSim_Story.set_index(pd.Index(title_list)).rename(columns=dict(zip(cosSim_Story.columns, title_list)))

###Rating pre-processing

In [ ]:
new_ratings_refined = rating[rating['movieId'].isin(movie_id_list)]

In [ ]:
new_ratings_refined = pd.merge(new_ratings_refined, movie, on='movieId', how='left')

In [ ]:
new_ratings_refined = new_ratings_refined[['userId', 'movieId', 'rating', 'title']]

In [ ]:
new_ratings_refined

,userId,movieId,rating,title
0,1,1,4.0,Toy Story
1,1,3,4.0,Grumpier Old Men
2,1,6,4.0,Heat
3,1,47,5.0,Seven (a.k.a. Se7en)
4,1,50,5.0,"Usual Suspects, The"
...,...,...,...,...
100694,610,166534,4.0,Split
100695,610,168248,5.0,John Wick: Chapter Two
100696,610,168250,5.0,Get Out
100697,610,168252,5.0,Logan


##Modeling

### 1. python

In [ ]:
from tqdm.notebook import tqdm
import numpy as np
def modeling(similarity_matrix, data):
    df_pred_all = pd.DataFrame()
    titles = title_list
    all_users = sorted(data['userId'].unique())
    n_titles = len(titles)

    for user in tqdm(all_users):
        idx = data[data['userId'] == user].index

        # 유사도
        watched_movies = data.loc[idx, 'movieId'].tolist()
        sub_sim_mat = similarity_matrix.loc[watched_movies]
        sub_sim_mat = sub_sim_mat.T.to_numpy()
        sim_N = np.sum(sub_sim_mat, axis=1) + 1


        # 평점 예측
        watched_movies_y = data.loc[idx, 'rating']
        watched_movies_y = np.array(watched_movies_y.tolist()).reshape(-1, 1)

        pred_y = np.matmul(sub_sim_mat, watched_movies_y).flatten() / sim_N

        users_list = [user] * n_titles
        cur_pred = pd.DataFrame(zip(users_list, titles, pred_y),
                                columns=['userId', 'title', 'pred_rating'])

        # 결과 기록
        df_pred_all = pd.concat([df_pred_all, cur_pred], axis=0)

    return df_pred_all

### 2. Spark

In [ ]:
from pyspark import SparkContext, SparkConf

def calculate_predictions(sub_sim_mat, watched_movies_y, sim_N):
    # Initialize SparkContext
    conf = SparkConf().setAppName("ColabApp").setMaster("local[*]")\
                      .set('spark.executor.memory', '4g')\
                      .set('spark.driver.memory', '4g')
    sc = SparkContext.getOrCreate(conf=conf)

    # Create RDDs for sub_sim_mat, watched_movies_y, and sim_N
    sub_sim_mat_rdd = sc.parallelize(sub_sim_mat, numSlices=4)
    watched_movies_y_rdd = sc.parallelize(watched_movies_y, numSlices=4)
    sim_N_bc = sc.broadcast(sim_N)

    # Calculate pred_y using Spark transformations and actions
    def calculate_pred_y(sub_sim_mat_row):
        return np.dot(sub_sim_mat_row, watched_movies_y) / sim_N_bc.value

    pred_y_rdd = sub_sim_mat_rdd.map(calculate_pred_y)
    pred_y = pred_y_rdd.collect()

    # Flatten the result if necessary
    pred_y_flat = np.array(pred_y).flatten()

    # Stop SparkContext
    sc.stop()

    return pred_y_flat


In [ ]:
def modeling_spark(similarity_matrix, data):
    df_pred_all = pd.DataFrame()
    titles = title_list
    all_users = sorted(data['userId'].unique())
    n_titles = len(titles)

    for user in tqdm(all_users):
        idx = data[data['userId'] == user].index

        # 유사도
        watched_movies = data.loc[idx, 'movieId'].tolist()
        sub_sim_mat = similarity_matrix.loc[watched_movies]
        sub_sim_mat = sub_sim_mat.T.to_numpy()
        sim_N = np.sum(sub_sim_mat, axis=1) + 1

        # 평점 예측
        watched_movies_y = data.loc[idx, 'rating']
        watched_movies_y = np.array(watched_movies_y.tolist()).reshape(-1, 1)

        # pred_y 계산 (PySpark)
        pred_y_spark = calculate_predictions(sub_sim_mat, watched_movies_y, sim_N)

        # pred_y를 NumPy 배열로 변환하고 계산
        pred_y = np.array(pred_y_spark)
        users_list = [user] * n_titles
        cur_pred = pd.DataFrame(zip(users_list, titles, pred_y),
                                columns=['userId', 'title', 'pred_rating'])

        # 결과 기록
        df_pred_all = pd.concat([df_pred_all, cur_pred], axis=0)

    return df_pred_all

In [ ]:
cosSim_All_id

,1,2,3,4,5,6,7,8,9,10,...,191005,193565,193567,193571,193573,193581,193583,193585,193587,193609
1,1.000000,0.876327,0.716804,0.457971,0.617727,0.508218,0.395936,0.849832,0.447652,0.434804,...,0.323645,0.545659,0.465674,0.377197,0.548296,0.508439,0.266175,0.616943,0.506044,0.865413
2,0.875729,1.000000,0.665633,0.475699,0.569404,0.553243,0.416010,0.889508,0.543152,0.444473,...,0.311305,0.410299,0.386816,0.305902,0.479891,0.448355,0.223821,0.636172,0.454245,0.743381
3,0.718801,0.669587,1.000000,0.630407,0.575003,0.569360,0.525845,0.628621,0.443755,0.446823,...,0.298588,0.500797,0.452085,0.348682,0.422717,0.612429,0.260799,0.675633,0.519121,0.855869
4,0.427382,0.445211,0.571714,1.000000,0.762320,0.806226,0.612316,0.391407,0.698074,0.528154,...,0.483952,0.181812,0.249821,0.554674,0.222749,0.170516,0.466419,0.512938,0.178643,0.526398
5,0.635450,0.591340,0.591828,0.862821,1.000000,0.820115,0.646798,0.573423,0.730685,0.557048,...,0.546175,0.286394,0.252738,0.584853,0.353173,0.272550,0.535068,0.424955,0.236189,0.615567
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
193581,0.538648,0.484746,0.633665,0.217032,0.284066,0.298944,0.353738,0.415349,0.276585,0.415184,...,0.632366,0.844378,0.682152,0.554198,0.685501,1.000000,0.678158,0.596529,0.940878,0.601799
193583,0.274540,0.242644,0.264844,0.475825,0.475974,0.472470,0.416743,0.142612,0.429535,0.465356,...,0.829749,0.566794,0.406734,0.738282,0.488074,0.595529,1.000000,0.307969,0.623458,0.313496
193585,0.552452,0.571494,0.601445,0.496902,0.352995,0.432765,0.224916,0.505252,0.294081,0.265834,...,0.391910,0.537624,0.623860,0.486067,0.477113,0.503947,0.272358,1.000000,0.563699,0.596747
193587,0.468306,0.426794,0.475788,0.178339,0.200559,0.297555,0.183868,0.393537,0.292695,0.316790,...,0.611707,0.827140,0.705058,0.482613,0.709672,0.829547,0.616559,0.577450,1.000000,0.525424


##Rating prediction

###Cosine Similarity

####all

In [ ]:
df_pred_cos_all_sp = pd.read_csv(path + 'df_pred_cos_all_sp.csv')

In [ ]:
df_pred_cos_all = df_pred_cos_all_sp

In [ ]:
df_pred_cos_all = modeling(cosSim_All_id,new_ratings_refined)

NameError: ignored

####Story

In [ ]:
df_pred_cos_story_sp = pd.read_csv(path + 'df_pred_cos_story_sp.csv')

In [ ]:
df_pred_cos_story=modeling(cosSim_Story_id,new_ratings_refined)

  0%|          | 0/610 [00:00<?, ?it/s]

####Genre

In [ ]:
df_pred_cos_genre_sp = pd.read_csv(path + 'df_pred_cos_genre_sp.csv')

In [ ]:
df_pred_cos_genre=modeling(cosSim_Genres_id,new_ratings_refined)

  0%|          | 0/610 [00:00<?, ?it/s]

####Title

In [ ]:
df_pred_cos_title_sp = pd.read_csv(path + 'df_pred_cos_title_sp.csv')

In [ ]:
df_pred_cos_title=modeling(cosSim_Title_id,new_ratings_refined)

  0%|          | 0/610 [00:00<?, ?it/s]

####Tag

In [ ]:
df_pred_cos_tag_sp = pd.read_csv(path + 'df_pred_cos_tag_sp.csv')

In [ ]:
df_pred_cos_tag=modeling(cosSim_Tag_id,new_ratings_refined)

  0%|          | 0/610 [00:00<?, ?it/s]

####Story * Title * Genre * Tag


###Euclidean Similarity

####All

In [ ]:
df_pred_euc_all_sp = pd.read_csv(path + 'df_pred_euc_all_sp.csv')

In [ ]:
df_pred_euc_all = df_pred_euc_all_sp

In [ ]:
df_pred_euc_all=modeling(EucSim_All_id,new_ratings_refined)

####Story

In [ ]:
df_pred_euc_story_sp = pd.read_csv(path + 'df_pred_euc_story_sp.csv')

In [ ]:
df_pred_euc_story = df_pred_euc_story_sp

In [ ]:
df_pred_euc_story=modeling(EucSim_Story_id,new_ratings_refined)

  0%|          | 0/610 [00:00<?, ?it/s]

####Genre

In [ ]:
df_pred_euc_genre_sp = pd.read_csv(path + 'df_pred_euc_genre_sp.csv')

FileNotFoundError: ignored

In [ ]:
df_pred_euc_genre = df_pred_euc_genre_sp

NameError: ignored

In [ ]:
df_pred_euc_genre=modeling(EucSim_Genres_id,new_ratings_refined)

####Title

In [ ]:
df_pred_euc_title_sp = pd.read_csv(path + 'df_pred_euc_title_sp.csv')

In [ ]:
df_pred_euc_title = df_pred_euc_title_sp

In [ ]:
df_pred_euc_title=modeling(EucSim_Title_id,new_ratings_refined)

  0%|          | 0/610 [00:00<?, ?it/s]

####Tag

In [ ]:
df_pred_cos_tag_sp = pd.read_csv(path + 'df_pred_euc_tag_sp.csv')

In [ ]:
df_pred_euc_tag = df_pred_cos_tag_sp

In [ ]:
df_pred_euc_tag=modeling(EucSim_Tag_id,new_ratings_refined)

  0%|          | 0/610 [00:00<?, ?it/s]

####Story * Title * Genre * Tag

## Evaluation

In [ ]:
from sklearn.model_selection import train_test_split

_, test_data = train_test_split(new_ratings_refined[['userId', 'title', 'rating']],
                test_size=20000, random_state=1234, stratify=new_ratings_refined['userId']) # 2만개 샘플링
test_data

,userId,title,rating
48238,313,Star Wars: Episode VI - Return of the Jedi,3.0
36923,249,Push,3.5
36015,246,Arizona Dream,4.0
64042,414,Finding Nemo,4.5
85364,555,Batman Returns,3.0
...,...,...,...
556,5,Pinocchio,5.0
10198,66,Beverly Hills Cop,3.0
53959,356,"Bourne Supremacy, The",4.0
35434,239,28 Days Later,5.0


### MAE, MSE, RMSE

In [ ]:
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
def calculate_errors(test_data):
    true_y = np.array(test_data['rating'])
    pred_y = np.array(test_data['pred_rating'])

    mae = mean_absolute_error(y_true=true_y, y_pred=pred_y)
    mse = mean_squared_error(y_true=true_y, y_pred=pred_y)
    rmse = np.sqrt(mse)

    calculate_errors = []
    print(f"MAE  : {str(round(mae, 2))}")
    print(f"MSE  : {str(round(mse, 2))}")
    print(f"RMSE : {str(round(rmse, 2))}")

    calculate_errors.append(mae)
    calculate_errors.append(mse)
    calculate_errors.append(rmse)

    return calculate_errors

 ### Precision, Recall

In [ ]:
import numpy as np

def get_precision_recall_f1(test_data):
    threshold = 3
    true_y = np.array(test_data['rating'])
    pred_y = np.array(test_data['pred_rating'])

    # True positives and negatives
    trues = 1 * (true_y >= threshold)
    pred_trues = 1 * (pred_y >= threshold)

    # Calculate True Positives
    true_positive = sum(trues & pred_trues)

    # Precision Calculation
    predicted_positives = sum(pred_trues)
    precision = true_positive / predicted_positives if predicted_positives else 0

    # Recall Calculation
    false_negatives = sum(trues & ~pred_trues)
    recall = true_positive / (true_positive + false_negatives) if (true_positive + false_negatives) else 0

    # F1-Score Calculation
    f1_score = 2 * (precision * recall) / (precision + recall) if (precision + recall) else 0

    print(f"Precision: {round(precision, 2)}")
    print(f"Recall: {round(recall, 2)}")
    print(f"F1-Score: {round(f1_score, 2)}")

    result = []
    result.append(precision)
    result.append(recall)
    result.append(f1_score)

    return result


### NDCG

In [ ]:
def dcg_at_k(relevances, k=10):
    relevances = np.asfarray(relevances)[:k]
    n_relevances = len(relevances)
    if n_relevances == 0:
        return 0.

    discounts = np.log2(np.arange(n_relevances) + 2)

    return np.sum(relevances / discounts)

def ndcg_at_k(test_data, k=10):
    true_y = np.array(test_data['rating'])
    pred_y = np.array(test_data['pred_rating'])

    idcg = dcg_at_k(sorted(true_y, reverse=True), k)
    dcg = dcg_at_k(pred_y, k)
    ndcg = dcg / idcg if idcg > 0 else 0

    # 결과 출력
    print(f"NDCG Score : {round(ndcg, 2)}")

    return ndcg

#### Cosine Similarity

#####All

In [ ]:
test_data_cos_all = pd.merge(test_data, df_pred_cos_all, on=['userId', 'title'], how='left') # userid와 title을 기준으로 합치기
test_data_cos_all

,userId,title,rating,pred_rating
0,313,Star Wars: Episode VI - Return of the Jedi,3.0,3.027375
1,249,Push,3.5,2.997994
2,246,Arizona Dream,4.0,3.538682
3,414,Finding Nemo,4.5,3.148872
4,555,Batman Returns,3.0,3.594758
...,...,...,...,...
21471,5,Pinocchio,5.0,4.865209
21472,66,Beverly Hills Cop,3.0,3.590364
21473,356,"Bourne Supremacy, The",4.0,3.672244
21474,239,28 Days Later,5.0,3.013930


In [ ]:
print("Cosine_simility All \n")
cos_all = calculate_errors(test_data_cos_all)
cos_all.append(get_precision_recall_f1(test_data_cos_all))
cos_all.append(ndcg_at_k(test_data_cos_all))

Cosine_simility All 

MAE  : 0.92
MSE  : 1.28
RMSE : 1.13
Precision: 0.86
Recall: 0.59
F1-Score: 0.7
NDCG Score : 0.64


##### Story

In [ ]:
test_data_cos_story = pd.merge(test_data, df_pred_cos_story, on=['userId', 'title'], how='left') # userid와 title을 기준으로 합치기
test_data_cos_story

,userId,title,rating,pred_rating
0,313,Star Wars: Episode VI - Return of the Jedi,3.0,3.419626
1,249,Push,3.5,3.708356
2,246,Arizona Dream,4.0,4.227381
3,414,Finding Nemo,4.5,3.396308
4,555,Batman Returns,3.0,3.445773
...,...,...,...,...
21471,5,Pinocchio,5.0,3.581021
21472,66,Beverly Hills Cop,3.0,4.006366
21473,356,"Bourne Supremacy, The",4.0,3.984139
21474,239,28 Days Later,5.0,4.025448


In [ ]:
print("Cosine_simility Story \n")
cos_story = calculate_errors(test_data_cos_story)
cos_story.append(get_precision_recall_f1(test_data_cos_story))
cos_story.append(ndcg_at_k(test_data_cos_story))

Cosine_simility Story 

MAE  : 0.73
MSE  : 0.87
RMSE : 0.93
Precision: 0.86
Recall: 0.91
F1-Score: 0.88
NDCG Score : 0.73


##### Genre

In [ ]:
test_data_cos_genre = pd.merge(test_data, df_pred_cos_genre, on=['userId', 'title'], how='left') # userid와 title을 기준으로 합치기
test_data_cos_genre

,userId,title,rating,pred_rating
0,313,Star Wars: Episode VI - Return of the Jedi,3.0,3.430877
1,249,Push,3.5,3.666048
2,246,Arizona Dream,4.0,4.213205
3,414,Finding Nemo,4.5,3.307466
4,555,Batman Returns,3.0,3.420063
...,...,...,...,...
21471,5,Pinocchio,5.0,3.354067
21472,66,Beverly Hills Cop,3.0,3.987860
21473,356,"Bourne Supremacy, The",4.0,3.906736
21474,239,28 Days Later,5.0,3.888930


In [ ]:
print("Cosine_simility Genre \n")
cos_genre = calculate_errors(test_data_cos_genre)
cos_genre.append(get_precision_recall_f1(test_data_cos_genre))
cos_genre.append(ndcg_at_k(test_data_cos_genre))

Cosine_simility Genre 

MAE  : 0.71
MSE  : 0.82
RMSE : 0.9
Precision: 0.87
Recall: 0.88
F1-Score: 0.88
NDCG Score : 0.73


##### Title

In [ ]:
test_data_cos_title= pd.merge(test_data, df_pred_cos_title, on=['userId', 'title'], how='left') # userid와 title을 기준으로 합치기
test_data_cos_title

,userId,title,rating,pred_rating
0,313,Star Wars: Episode VI - Return of the Jedi,3.0,3.421079
1,249,Push,3.5,3.670435
2,246,Arizona Dream,4.0,4.251560
3,414,Finding Nemo,4.5,3.380272
4,555,Batman Returns,3.0,3.431706
...,...,...,...,...
21471,5,Pinocchio,5.0,3.518909
21472,66,Beverly Hills Cop,3.0,4.023823
21473,356,"Bourne Supremacy, The",4.0,4.014433
21474,239,28 Days Later,5.0,3.970287


In [ ]:
print("Cosine_simility Title \n")
cos_title = calculate_errors(test_data_cos_title)
cos_title = get_precision_recall_f1(test_data_cos_title)
cos_title.append(ndcg_at_k(test_data_cos_title))

Cosine_simility Title 

MAE  : 0.73
MSE  : 0.87
RMSE : 0.93
Precision: 0.86
Recall: 0.91
F1-Score: 0.88
NDCG Score : 0.73


##### Tag

In [ ]:
test_data_cos_tag = pd.merge(test_data, df_pred_cos_tag, on=['userId', 'title'], how='left') # userid와 title을 기준으로 합치기
test_data_cos_tag

,userId,title,rating,pred_rating
0,313,Star Wars: Episode VI - Return of the Jedi,3.0,3.402916
1,249,Push,3.5,3.698026
2,246,Arizona Dream,4.0,4.208695
3,414,Finding Nemo,4.5,3.392270
4,555,Batman Returns,3.0,3.434853
...,...,...,...,...
21471,5,Pinocchio,5.0,3.543437
21472,66,Beverly Hills Cop,3.0,3.999580
21473,356,"Bourne Supremacy, The",4.0,3.992357
21474,239,28 Days Later,5.0,4.003369


In [ ]:
print("Cosine_simility Tag \n")
cos_tag = calculate_errors(test_data_cos_tag)
cos_tag.append(get_precision_recall_f1(test_data_cos_tag))
cos_tag.append(ndcg_at_k(test_data_cos_tag))

Cosine_simility Tag 

MAE  : 0.78
MSE  : 1.1
RMSE : 1.05
Precision: 0.86
Recall: 0.89
F1-Score: 0.87
NDCG Score : 0.73


#### Euclidean Similarity

##### All

In [ ]:
test_data_euc_all= pd.merge(test_data, df_pred_euc_all, on=['userId', 'title'], how='left') # userid와 title을 기준으로 합치기
test_data_euc_all

,userId,title,rating,pred_rating
0,313,Star Wars: Episode VI - Return of the Jedi,3.0,3.927208
1,249,Push,3.5,4.830012
2,246,Arizona Dream,4.0,3.911656
3,414,Finding Nemo,4.5,3.774880
4,555,Batman Returns,3.0,3.652823
...,...,...,...,...
21471,5,Pinocchio,5.0,2.256649
21472,66,Beverly Hills Cop,3.0,4.600017
21473,356,"Bourne Supremacy, The",4.0,4.477074
21474,239,28 Days Later,5.0,4.179060


In [ ]:
print("Euclidean Similarity All \n")
euc_all = calculate_errors(test_data_euc_all)
euc_all.append(get_precision_recall_f1(test_data_euc_all))
euc_all.append(ndcg_at_k(test_data_euc_all))

Euclidean Similarity All 

MAE  : 0.95
MSE  : 1.47
RMSE : 1.21
Precision: 0.83
Recall: 0.89
F1-Score: 0.86
NDCG Score : 0.82


##### Story

In [ ]:
test_data_euc_story = pd.merge(test_data, df_pred_euc_story, on=['userId', 'title'], how='left') # userid와 title을 기준으로 합치기
test_data_euc_story

,userId,title,rating,pred_rating
0,313,Star Wars: Episode VI - Return of the Jedi,3.0,3.997916
1,249,Push,3.5,3.549206
2,246,Arizona Dream,4.0,4.562285
3,414,Finding Nemo,4.5,3.845302
4,555,Batman Returns,3.0,3.300467
...,...,...,...,...
21471,5,Pinocchio,5.0,2.309630
21472,66,Beverly Hills Cop,3.0,3.615569
21473,356,"Bourne Supremacy, The",4.0,3.066924
21474,239,28 Days Later,5.0,4.565764


In [ ]:
print("Euclidean Similarity stroy \n")
euc_story = calculate_errors(test_data_euc_story)
euc_story.append(get_precision_recall_f1(test_data_euc_story))
euc_story.append(ndcg_at_k(test_data_euc_story))

Euclidean Similarity stroy 

MAE  : 0.87
MSE  : 1.26
RMSE : 1.12
Precision: 0.86
Recall: 0.74
F1-Score: 0.79
NDCG Score : 0.77


##### Genre

In [ ]:
test_data_euc_genre = pd.merge(test_data, df_pred_euc_genre, on=['userId', 'title'], how='left') # userid와 title을 기준으로 합치기
test_data_euc_genre

NameError: ignored

In [ ]:
print("Euclidean Similarity genre \n")
euc_genere = calculate_errors(test_data_euc_genre)
euc_genere.append(get_precision_recall_f1(test_data_euc_genre))
euc_genere.append(ndcg_at_k(test_data_euc_genre))

##### Title

In [ ]:
test_data_euc_title = pd.merge(test_data, df_pred_euc_title, on=['userId', 'title'], how='left') # userid와 title을 기준으로 합치기
test_data_euc_title

In [ ]:
print("Euclidean Similarity title \n")
euc_title = calculate_errors(test_data_euc_title)
euc_title.append(get_precision_recall_f1(test_data_euc_title))
euc_title.append(ndcg_at_k(test_data_euc_title))

Euclidean Similarity title 



NameError: ignored

##### Tag

In [ ]:
test_data_euc_tag = pd.merge(test_data, df_pred_euc_tag, on=['userId', 'title'], how='left') # userid와 title을 기준으로 합치기
test_data_euc_tag

,userId,title,rating,pred_rating
0,313,Star Wars: Episode VI - Return of the Jedi,3.0,4.091558
1,249,Push,3.5,4.418665
2,246,Arizona Dream,4.0,5.348602
3,414,Finding Nemo,4.5,3.595484
4,555,Batman Returns,3.0,3.067748
...,...,...,...,...
21471,5,Pinocchio,5.0,0.894177
21472,66,Beverly Hills Cop,3.0,4.801323
21473,356,"Bourne Supremacy, The",4.0,5.512336
21474,239,28 Days Later,5.0,5.064312


In [ ]:
print("Euclidean Similarity tag \n")
euc_tag = calculate_errors(test_data_euc_tag)
euc_tag.append(get_precision_recall_f1(test_data_euc_tag))
euc_tag.append(ndcg_at_k(test_data_euc_tag))

Euclidean Similarity tag 

MAE  : 1.11
MSE  : 1.94
RMSE : 1.39
Precision: 0.84
Recall: 0.75
F1-Score: 0.79
NDCG Score : 0.86


##Recomendation

In [ ]:
user_id = 1

###Cosine Similarity

####All

In [ ]:
# 사용자가 안본 영화 중 예측 평점 높은 10개 추천
def get_recomendation_cos_all(user_id):
    user_mov = new_ratings_refined[new_ratings_refined['userId'] == user_id]
    user_mov_pred = df_pred_cos_all[df_pred_cos_all['userId'] == user_id]
    user_mov = pd.merge(user_mov, user_mov_pred, on=['userId', 'title'], how='right')

    # 사용자가 아직 안 본 영화
    movie_candidate = user_mov[user_mov['movieId'].isnull()]
    movie_candidate = movie_candidate.sort_values(by='pred_rating', ascending=False)[:10]

    return movie_candidate['title'].tolist()

In [ ]:
get_recomendation_cos_all(user_id)

['Secret of Kells, The',
 'Blood: The Last Vampire',
 'The Green Inferno',
 'Counterfeiters, The (Die Fälscher)',
 'Aileen: Life and Death of a Serial Killer',
 'The Monkey King',
 'In the Mood For Love (Fa yeung nin wa)',
 'Proposition, The',
 'City of Lost Souls, The (Hyôryuu-gai)',
 'Darkest Hour, The']

####Story

In [ ]:
# 사용자가 안본 영화 중 예측 평점 높은 10개 추천
def get_recomendation_cos_story(user_id):
    user_mov = new_ratings_refined[new_ratings_refined['userId'] == user_id]
    user_mov_pred = df_pred_cos_story[df_pred_cos_story['userId'] == user_id]
    user_mov = pd.merge(user_mov, user_mov_pred, on=['userId', 'title'], how='right')

    # 사용자가 아직 안 본 영화
    movie_candidate = user_mov[user_mov['movieId'].isnull()]
    movie_candidate = movie_candidate.sort_values(by='pred_rating', ascending=False)[:10]

    return movie_candidate['title'].tolist()

In [ ]:
get_recomendation_cos_story(user_id)

####Genre

In [ ]:
# 사용자가 안본 영화 중 예측 평점 높은 10개 추천
def get_recomendation_cos_genre(user_id):
    user_mov = new_ratings_refined[new_ratings_refined['userId'] == user_id]
    user_mov_pred = df_pred_cos_genre[df_pred_cos_genre['userId'] == user_id]
    user_mov = pd.merge(user_mov, user_mov_pred, on=['userId', 'title'], how='right')

    # 사용자가 아직 안 본 영화
    movie_candidate = user_mov[user_mov['movieId'].isnull()]
    movie_candidate = movie_candidate.sort_values(by='pred_rating', ascending=False)[:10]

    return movie_candidate['title'].tolist()

In [ ]:
get_recomendation_cos_genre(user_id)

####Title

In [ ]:
# 사용자가 안본 영화 중 예측 평점 높은 10개 추천
def get_recomendation_cos_title(user_id):
    user_mov = new_ratings_refined[new_ratings_refined['userId'] == user_id]
    user_mov_pred = df_pred_cos_title[df_pred_cos_title['userId'] == user_id]
    user_mov = pd.merge(user_mov, user_mov_pred, on=['userId', 'title'], how='right')

    # 사용자가 아직 안 본 영화
    movie_candidate = user_mov[user_mov['movieId'].isnull()]
    movie_candidate = movie_candidate.sort_values(by='pred_rating', ascending=False)[:10]

    return movie_candidate['title'].tolist()

In [ ]:
get_recomendation_cos_title(user_id)

####Tag

In [ ]:
# 사용자가 안본 영화 중 예측 평점 높은 10개 추천
def get_recomendation_cos_tag(user_id):
    user_mov = new_ratings_refined[new_ratings_refined['userId'] == user_id]
    user_mov_pred = df_pred_cos_tag[df_pred_cos_tag['userId'] == user_id]
    user_mov = pd.merge(user_mov, user_mov_pred, on=['userId', 'title'], how='right')

    # 사용자가 아직 안 본 영화
    movie_candidate = user_mov[user_mov['movieId'].isnull()]
    movie_candidate = movie_candidate.sort_values(by='pred_rating', ascending=False)[:10]

    return movie_candidate['title'].tolist()

In [ ]:
get_recomendation_cos_tag(user_id)

###Euclidean Similarity

####All

In [ ]:
# 사용자가 안본 영화 중 예측 평점 높은 10개 추천
def get_recomendation_euc_all(user_id):
    user_mov = new_ratings_refined[new_ratings_refined['userId'] == user_id]
    user_mov_pred = df_pred_euc_all[df_pred_euc_all['userId'] == user_id]
    user_mov = pd.merge(user_mov, user_mov_pred, on=['userId', 'title'], how='right')

    # 사용자가 아직 안 본 영화
    movie_candidate = user_mov[user_mov['movieId'].isnull()]
    movie_candidate = movie_candidate.sort_values(by='pred_rating', ascending=False)[:10]

    return movie_candidate['title'].tolist()

In [ ]:
get_recomendation_euc_all(user_id)

['Major League II',
 'Chasers',
 'Bob Roberts',
 'Sister Act 2: Back in the Habit',
 'Major League: Back to the Minors',
 'House Party 3',
 'Destiny Turns on the Radio',
 'Fled',
 'Schizopolis',
 'New Waterford Girl']

####Story

In [ ]:
# 사용자가 안본 영화 중 예측 평점 높은 10개 추천
def get_recomendation_euc_story(user_id):
    user_mov = new_ratings_refined[new_ratings_refined['userId'] == user_id]
    user_mov_pred = df_pred_euc_story[df_pred_euc_story['userId'] == user_id]
    user_mov = pd.merge(user_mov, user_mov_pred, on=['userId', 'title'], how='right')

    # 사용자가 아직 안 본 영화
    movie_candidate = user_mov[user_mov['movieId'].isnull()]
    movie_candidate = movie_candidate.sort_values(by='pred_rating', ascending=False)[:10]

    return movie_candidate['title'].tolist()

In [ ]:
get_recomendation_euc_story(user_id)

['Self-criticism of a Bourgeois Dog',
 "Big Momma's House",
 'Rainmaker, The',
 'Oz the Great and Powerful',
 'Don Camillo in Moscow',
 'Man Who Knew Too Much, The',
 'Fled',
 'Dinotopia',
 "Fool's Gold",
 'RocknRolla']

####Genre

In [ ]:
# 사용자가 안본 영화 중 예측 평점 높은 10개 추천
def get_recomendation_euc_genre(user_id):
    user_mov = new_ratings_refined[new_ratings_refined['userId'] == user_id]
    user_mov_pred = df_pred_euc_genre[df_pred_euc_genre['userId'] == user_id]
    user_mov = pd.merge(user_mov, user_mov_pred, on=['userId', 'title'], how='right')

    # 사용자가 아직 안 본 영화
    movie_candidate = user_mov[user_mov['movieId'].isnull()]
    movie_candidate = movie_candidate.sort_values(by='pred_rating', ascending=False)[:10]

    return movie_candidate['title'].tolist()

In [ ]:
get_recomendation_euc_genre(user_id)

NameError: ignored

####Title

In [ ]:
# 사용자가 안본 영화 중 예측 평점 높은 10개 추천
def get_recomendation_euc_title(user_id):
    user_mov = new_ratings_refined[new_ratings_refined['userId'] == user_id]
    user_mov_pred = df_pred_euc_title[df_pred_euc_title['userId'] == user_id]
    user_mov = pd.merge(user_mov, user_mov_pred, on=['userId', 'title'], how='right')

    # 사용자가 아직 안 본 영화
    movie_candidate = user_mov[user_mov['movieId'].isnull()]
    movie_candidate = movie_candidate.sort_values(by='pred_rating', ascending=False)[:10]

    return movie_candidate['title'].tolist()

In [ ]:
get_recomendation_euc_title(user_id)

['AVPR: Aliens vs. Predator - Requiem',
 "Happenstance (Battement d'ailes du papillon, Le)",
 'Dark Water (Honogurai mizu no soko kara)',
 "King Solomon's Mines",
 'Wet Hot American Summer',
 'Wicked City (Yôjû toshi)',
 'Beijing Bicycle (Shiqi sui de dan che)',
 'School Daze',
 'No Holds Barred',
 "Mr. Popper's Penguins"]

####Tag

In [ ]:
# 사용자가 안본 영화 중 예측 평점 높은 10개 추천
def get_recomendation_euc_tag(user_id):
    user_mov = new_ratings_refined[new_ratings_refined['userId'] == user_id]
    user_mov_pred = df_pred_euc_tag[df_pred_euc_tag['userId'] == user_id]
    user_mov = pd.merge(user_mov, user_mov_pred, on=['userId', 'title'], how='right')

    # 사용자가 아직 안 본 영화
    movie_candidate = user_mov[user_mov['movieId'].isnull()]
    movie_candidate = movie_candidate.sort_values(by='pred_rating', ascending=False)[:10]

    return movie_candidate['title'].tolist()

In [ ]:
get_recomendation_euc_title(user_id)

['AVPR: Aliens vs. Predator - Requiem',
 "Happenstance (Battement d'ailes du papillon, Le)",
 'Dark Water (Honogurai mizu no soko kara)',
 "King Solomon's Mines",
 'Wet Hot American Summer',
 'Wicked City (Yôjû toshi)',
 'Beijing Bicycle (Shiqi sui de dan che)',
 'School Daze',
 'No Holds Barred',
 "Mr. Popper's Penguins"]

##Develop

### Pred_Rating data round

In [ ]:
test_data_cos_all_round = pd.merge(test_data, df_pred_cos_all, on=['userId', 'title'], how='left') # userid와 title을 기준으로 합치기
test_data_cos_all_round

,userId,title,rating,pred_rating
0,313,Star Wars: Episode VI - Return of the Jedi,3.0,3.027375
1,249,Push,3.5,2.997994
2,246,Arizona Dream,4.0,3.538682
3,414,Finding Nemo,4.5,3.148872
4,555,Batman Returns,3.0,3.594758
...,...,...,...,...
21471,5,Pinocchio,5.0,4.865209
21472,66,Beverly Hills Cop,3.0,3.590364
21473,356,"Bourne Supremacy, The",4.0,3.672244
21474,239,28 Days Later,5.0,3.013930


In [ ]:
test_data_cos_all_round['pred_rating'] = test_data_cos_all_round['pred_rating'].round(1)

In [ ]:
print("Cosine_simility All_Round \n")
cos_all = calculate_errors(test_data_cos_all_round)
cos_all.append(get_precision_recall_f1(test_data_cos_all_round))
cos_all.append(ndcg_at_k(test_data_cos_all_round))

Cosine_simility All 

MAE  : 0.91
MSE  : 1.35
RMSE : 1.16
Precision: 0.83
Recall: 0.9
F1-Score: 0.86
NDCG Score : 0.65


### Coverage Set

In [ ]:
test_data_cos_all_Coverage = pd.merge(test_data, df_pred_cos_all, on=['userId', 'title'], how='left') # userid와 title을 기준으로 합치기
test_data_cos_all_Coverage

,userId,title,rating,pred_rating
0,313,Star Wars: Episode VI - Return of the Jedi,3.0,3.027375
1,249,Push,3.5,2.997994
2,246,Arizona Dream,4.0,3.538682
3,414,Finding Nemo,4.5,3.148872
4,555,Batman Returns,3.0,3.594758
...,...,...,...,...
21471,5,Pinocchio,5.0,4.865209
21472,66,Beverly Hills Cop,3.0,3.590364
21473,356,"Bourne Supremacy, The",4.0,3.672244
21474,239,28 Days Later,5.0,3.013930


In [ ]:
threshold = 3
filtered_data = test_data_cos_all_Coverage[(test_data_cos_all_Coverage['rating'] > threshold) &
                                           (test_data_cos_all_Coverage['pred_rating'] > threshold)]
filtered_data

,userId,title,rating,pred_rating
2,246,Arizona Dream,4.0,3.538682
3,414,Finding Nemo,4.5,3.148872
6,246,Forrest Gump,3.5,3.272620
7,221,It's a Wonderful Life,5.0,3.130157
9,201,So I Married an Axe Murderer,5.0,3.929419
...,...,...,...,...
21467,450,North by Northwest,4.0,3.116578
21470,5,Pinocchio,5.0,4.420902
21471,5,Pinocchio,5.0,4.865209
21473,356,"Bourne Supremacy, The",4.0,3.672244


In [ ]:
print("Cosine_simility All_filtered \n")
cos_all = calculate_errors(filtered_data)
cos_all.append(get_precision_recall_f1(filtered_data))
cos_all.append(ndcg_at_k(filtered_data))

Cosine_simility All_filtered 

MAE  : 0.75
MSE  : 0.83
RMSE : 0.91
Precision: 1.0
Recall: 1.0
F1-Score: 1.0
NDCG Score : 0.69


###Round + Coverage Set

In [ ]:
test_data_cos_all_Coverage_Round = pd.merge(test_data, df_pred_cos_all, on=['userId', 'title'], how='left') # userid와 title을 기준으로 합치기
test_data_cos_all_Coverage_Round

,userId,title,rating,pred_rating
0,313,Star Wars: Episode VI - Return of the Jedi,3.0,3.027375
1,249,Push,3.5,2.997994
2,246,Arizona Dream,4.0,3.538682
3,414,Finding Nemo,4.5,3.148872
4,555,Batman Returns,3.0,3.594758
...,...,...,...,...
21471,5,Pinocchio,5.0,4.865209
21472,66,Beverly Hills Cop,3.0,3.590364
21473,356,"Bourne Supremacy, The",4.0,3.672244
21474,239,28 Days Later,5.0,3.013930


In [ ]:
threshold = 3
Round_filtered_data = test_data_cos_all_Coverage_Round[(test_data_cos_all_Coverage_Round['rating'] > threshold) &
                                           (test_data_cos_all_Coverage_Round['pred_rating'] > threshold)]
Round_filtered_data

,userId,title,rating,pred_rating
2,246,Arizona Dream,4.0,3.538682
3,414,Finding Nemo,4.5,3.148872
6,246,Forrest Gump,3.5,3.272620
7,221,It's a Wonderful Life,5.0,3.130157
9,201,So I Married an Axe Murderer,5.0,3.929419
...,...,...,...,...
21467,450,North by Northwest,4.0,3.116578
21470,5,Pinocchio,5.0,4.420902
21471,5,Pinocchio,5.0,4.865209
21473,356,"Bourne Supremacy, The",4.0,3.672244


In [ ]:
print("Cosine_simility All_filtered \n")
cos_all = calculate_errors(Round_filtered_data)
cos_all.append(get_precision_recall_f1(Round_filtered_data))
cos_all.append(ndcg_at_k(Round_filtered_data))

Cosine_simility All_filtered 

MAE  : 0.75
MSE  : 0.83
RMSE : 0.91
Precision: 1.0
Recall: 1.0
F1-Score: 1.0
NDCG Score : 0.69


## Supplement

### ALS + All_dot Hybrid

In [ ]:
!pip install pyspark
!curl -L -o "/usr/local/lib/python3.6/dist-packages/pyspark/jars/graphframes-0.8.2-spark3.2-s_2.12.jar" http://dl.bintray.com/spark-packages/maven/graphframes/graphframes/0.8.2-spark3.2-s_2.12/graphframes-0.8.2-spark3.2-s_2.12.jar
!apt-get install openjdk-8-jdk-headless -qq

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   164  100   164    0     0   6472      0 --:--:-- --:--:-- --:--:--  6560
100   146  100   146    0     0   1162      0 --:--:-- --:--:-- --:--:--  1162
curl: (23) Failure writing output to destination


In [ ]:
sc = pyspark.SparkContext()
spark = SparkSession.builder.getOrCreate()

In [ ]:
from pyspark.ml.recommendation import ALS
from pyspark.ml.evaluation import RegressionEvaluator
from sklearn.model_selection import train_test_split
from pyspark.ml.feature import StringIndexer
from pyspark.ml import Pipeline

spark_df_refined = spark.createDataFrame(new_ratings_refined)

# 사용자 ID와 아이템 ID(예: 영화 제목)를 숫자형으로 변환
user_indexer = StringIndexer(inputCol="userId", outputCol="user_id")
item_indexer = StringIndexer(inputCol="title", outputCol="item_id")
# Pipeline 생성
pipeline = Pipeline(stages=[user_indexer, item_indexer])
# 데이터 변환 실행
transformed_data = pipeline.fit(spark_df_refined).transform(spark_df_refined)
train_spark, test_spark = transformed_data.randomSplit([0.8, 0.2])


In [ ]:
train_spark.show()

+------+-------+------+--------------------+-------+-------+
|userId|movieId|rating|               title|user_id|item_id|
+------+-------+------+--------------------+-------+-------+
|     1|      1|   4.0|           Toy Story|  111.0|   11.0|
|     1|      3|   4.0|    Grumpier Old Men|  111.0|  429.0|
|     1|      6|   4.0|                Heat|  111.0|  131.0|
|     1|     47|   5.0|Seven (a.k.a. Se7en)|  111.0|   15.0|
|     1|     50|   5.0| Usual Suspects, The|  111.0|   14.0|
|     1|    101|   5.0|       Bottle Rocket|  111.0| 1105.0|
|     1|    110|   4.0|          Braveheart|  111.0|    7.0|
|     1|    151|   5.0|             Rob Roy|  111.0|  553.0|
|     1|    157|   5.0|      Canadian Bacon|  111.0| 1988.0|
|     1|    216|   5.0|       Billy Madison|  111.0|  457.0|
|     1|    223|   3.0|              Clerks|  111.0|  121.0|
|     1|    231|   5.0|Dumb & Dumber (Du...|  111.0|   62.0|
|     1|    235|   4.0|             Ed Wood|  111.0|  259.0|
|     1|    260|   5.0|S

In [ ]:
# 훈련 데이터와 테스트 데이터로 분할
rec = ALS(maxIter=10,
         regParam=0.01,
         userCol='user_id',
         itemCol='item_id',
         ratingCol='rating',
         nonnegative=True,
         coldStartStrategy='drop')
# ALS모델 학습
rec_model = rec.fit(train_spark)

# transform을 이용해 예측
als_pred = rec_model.transform(test_spark)

In [ ]:
# PySpark DataFrame을 Pandas DataFrame으로 변환
als_pred_pd = als_pred.toPandas()

# Pandas의 merge 함수를 사용해 병합
hybrid_df = pd.merge(als_pred_pd, df_pred_cos_all, on=['userId'], suffixes=('_als', '_cos'))

# 가중치를 적용하여 하이브리드 평점 생성
hybrid_df['hybrid_rating'] = 0.7 * hybrid_df['als_rating'] + 0.3 * hybrid_df['pred_rating']

# Display hybrid ratings
print(hybrid_df[['userId', 'hybrid_rating']])

NameError: ignored

In [ ]:
print("Cosine_simility All \n")
cos_all = calculate_errors(hybrid_df)
cos_all.append(get_precision_recall_f1(hybrid_df))
cos_all.append(ndcg_at_k(hybrid_df))

###SGD + All_dot Hybrid

In [ ]:
import numpy as np
import pandas as pd

class MatrixFactorization:
    def __init__(self, R, K, learning_rate, reg_param, epochs):
        """
        :param R: 사용자-아이템 평점 행렬 (데이터프레임)
        :param K: 잠재 요인의 차원
        :param learning_rate: 학습률
        :param reg_param: 정규화 파라미터
        :param epochs: 반복 횟수
        """
        self.R = R.values  # 데이터프레임을 numpy 배열로 변환
        self.num_users, self.num_items = self.R.shape
        self.K = K
        self.learning_rate = learning_rate
        self.reg_param = reg_param
        self.epochs = epochs

        # 잠재 요인 행렬 초기화
        self.P = np.random.normal(scale=1.0 / self.K, size=(self.num_users, self.K))
        self.Q = np.random.normal(scale=1.0 / self.K, size=(self.num_items, self.K))

    def train(self):
        for epoch in range(self.epochs):
            for i in range(self.num_users):
                for j in range(self.num_items):
                    if self.R[i][j] > 0:
                        eij = self.R[i][j] - np.dot(self.P[i, :], self.Q[j, :].T)
                        for k in range(self.K):
                            self.P[i][k] += self.learning_rate * (2 * eij * self.Q[j][k] - self.reg_param * self.P[i][k])
                            self.Q[j][k] += self.learning_rate * (2 * eij * self.P[i][k] - self.reg_param * self.Q[j][k])

            # 예측값을 특정 범위로 클리핑
            self.P = np.clip(self.P, 0, 5)
            self.Q = np.clip(self.Q, 0, 5)

            # 오차 계산
            error = self.calculate_error()
            print(f"Epoch {epoch + 1}/{self.epochs}, Error: {error}")

    def calculate_error(self):
        """
        모델의 현재 오차 계산
        """
        error = 0
        for i in range(self.num_users):
            for j in range(self.num_items):
                if self.R[i][j] > 0:
                    error += (self.R[i][j] - np.dot(self.P[i, :], self.Q[j, :].T))**2
                    for k in range(self.K):
                        error += (self.reg_param / 2) * (self.P[i][k]**2 + self.Q[j][k]**2)
        return error

    def predict(self):
        """
        학습된 모델을 사용하여 전체 사용자-아이템 평점 행렬 예측
        """
        return np.dot(self.P, self.Q.T)

# 잠재 요인의 차원, 학습률, 정규화 파라미터, 반복 횟수 등을 설정
K = 2
learning_rate = 0.01
reg_param = 0.02
epochs = 100

# 모델 생성 및 학습
mf = MatrixFactorization(spark_df_refined, K, learning_rate, reg_param, epochs)
mf.train()

# 예측 결과 출력
predicted_R = mf.predict()
predicted_R = np.clip(predicted_R, 0, 5)

In [ ]:
predicted_R_sgd = predicted_R.toPandas()

# Merge ALS and Spark predictions
hybrid_df_sgd = pd.merge(predicted_R_sgd, df_pred_euc_story, on=['userId'], suffixes=('_sgd', '_spark'))

# Apply weights to create hybrid predictions
hybrid_df_sgd['hybrid_rating'] = 0.7 * hybrid_df_sgd['sgd_rating'] + 0.3 * hybrid_df_sgd['pred_y']

# Display hybrid ratings
print(hybrid_df_sgd[['userId', 'hybrid_rating']])

In [ ]:
print("Cosine_simility All \n")
cos_all = calculate_errors(hybrid_df_sgd)
cos_all.append(get_precision_recall_f1(hybrid_df_sgd))
cos_all.append(ndcg_at_k(hybrid_df_sgd))